In [1]:
from gensim.models import KeyedVectors
import pandas as pd
import csv

In [2]:
wv_from_text = KeyedVectors.load_word2vec_format("./toastynews.vec", binary=False)

In [3]:
wv = wv_from_text

In [37]:
df = pd.read_csv('HKSL_src.csv')

with open('wordpos-1689192352.csv', newline='', encoding="utf-8") as posfile:
      tmp = csv.reader(posfile, delimiter=',')
      posrow = list(tmp)

df.insert(6,'pos','')

wordlist = df['Cantonese_Filtered'].tolist()

words = []
for i in range(len(posrow)):
    words.append(posrow[i][0])
    
# Loop through the Cantonese_Filtered column
for index, word in enumerate(wordlist):
    if not word or (word not in words):
        df.at[index, 'pos'] = 'NOTIN'
    else:
        # print(word)
        pos = []
        if words.count(word) > 1:
            for i in range(len(words)):
                if words[i] == word:
                # if i == word and posrow[words.index(i)][1].strip('[\'\']') not in df.at[index, 'pos']:
                    # df.at[index, 'pos'] += posrow[i][1].strip('[\'\']') + ','
                    tmp = posrow[i][1].strip('[\']')
                    [pos.append(i.strip(' \'')) for i in tmp.split(',') if i not in pos]
                    # pos.append(tmp.split('\', \''))
                    # pos = [*set(pos)]
        else:
            tmp = posrow[words.index(word)][1].strip('[\'],')
            # tmp.split('\', \'')
            # tmp = [i.strip('\'') for i in tmp]
            # pos.append(tmp.split('\', \''))
            # pos.append(tmp)
            # print(tmp.split('\', \''))
            # [print(i.strip(' \'')) for i in tmp.split(',')]
            [pos.append(i.strip(' \'')) for i in tmp.split(',')]
            
        # tmp = df.at[index, 'pos'].strip('\,')
        # poslist = []
        # for i in pos:
        #     poslist.append(i)
        # df.at[index, 'pos'] = poslist.sort()
        # df.at[index, 'pos'] = pos.sort()
        [i.strip('\' ') for i in pos]
        df.at[index, 'pos'] = sorted(pos)
        # print(f"{pos[0]} {type(pos[0])}  {pos[-1]} {type(pos[-1])}")

# Save the DataFrame to a new excel file
df.to_excel('HKSL_pos.xlsx', index=False)

In [12]:
# Read excel file as a pandas DataFrame
# df = pd.read_csv('HKSL_src.csv')
df = pd.read_csv('HKSL_pos.csv')

# Create a new column called "corr_cantonese"
df.insert(7,'5th','')
df.insert(8,'5th_pos','')
df.insert(9,'5th_filename','')
# # df.insert(7,'3rd','')
# df.insert(8,'5th','')
# df.insert(9,'10th','')
# df.insert(10,'-1','')

poslist = df['pos'].tolist()
wordlist = df['cantonese_orig'].tolist()
filenamelist = df['NewFileName'].tolist()

curpos = []
posindex = 0
# Loop through the Cantonese_Filtered column
for index, word in enumerate(wordlist):
    if poslist[index] == 'NOTIN' or poslist[index] == 'EXCLUDED':
        continue
    if poslist[index] != curpos:
        curpos = poslist[index]
        posindex = index
        
    # Check if word is empty or not in vocab
    if not word or (word not in wv.key_to_index):
        df.at[index, '5th'] = 'NOTIN'
        print(f"{word} is not in vocab")
        # df.at[index, 'cantonese_map_furthest'] = 'NOTIN'
    else:
        # Find the most similar word in the wordlist
        similarityList = []
        similarityWordList = []
        
        for innerword in wordlist:
            if innerword == word: 
                continue
            if not innerword or (innerword not in wv.key_to_index): 
                continue
            if poslist[wordlist.index(innerword)] != curpos:
                continue
            
            similarity = wv.similarity(word, innerword)
            
            similarityList.append(similarity)
            similarityList.sort(reverse=True)
            idx = similarityList.index(similarity)
            similarityWordList.insert(idx, innerword)   
            
            f = open(f'words/{word}.txt', "w", encoding="utf-8")
            # f.write(most_similar_word)
            f.write(f"{word} {poslist[index]}")
            f.write(word)
            f.write("\n")
            for i in range(len(similarityList)):
                f.write(str(similarityWordList[i]))
                f.write(" ")
                f.write(str(similarityList[i]))
                f.write(" ")
                f.write(poslist[wordlist.index(innerword)])
                if(i==0 or i==2 or i==4 or i==9 or i==len(similarityList)-1):
                    f.write(f" <-- {i+1}")
                f.write("\n")
            f.close()
            
                 
        # Fill in the corr_cantonese column with the most similar word found
        # df.at[index, '1st'] = similarityWordList[0]
        # df.at[index, 'mapped_NewFileName'] = filenamelist[index]
        # df.at[index, '3rd'] = similarityWordList[2]
        if len(similarityWordList) > 4:
            df.at[index, '5th'] = similarityWordList[4]
            df.at[index, '5th_pos'] = poslist[wordlist.index(similarityWordList[4])]
            df.at[index, '5th_filename'] = filenamelist[wordlist.index(similarityWordList[4])]
        else:
            df.at[index, '5th'] = len(similarityWordList)+1
        # df.at[index, '5th'] = similarityWordList[4]
        # df.at[index, '10th'] = similarityWordList[9]
        # df.at[index, '-1'] = similarityWordList[-1]

# Save the DataFrame to a new excel file
df.to_excel('HKSL_added.xlsx', index=False)


我的 is not in vocab
令人灰心 is not in vocab
碌落去 is not in vocab
遊船河 is not in vocab
用電傳打字機 is not in vocab
nan is not in vocab
背包旅行 is not in vocab
背包旅行 is not in vocab
手語名 is not in vocab
便衣警探 is not in vocab
自然手語 is not in vocab
交通津貼 is not in vocab
膳食津貼 is not in vocab
展能就業科 is not in vocab
nan is not in vocab
nan is not in vocab
臉色發白 is not in vocab
眼凸凸 is not in vocab
弱聽 is not in vocab
nan is not in vocab
玩滑板 is not in vocab
聽音樂 is not in vocab
玩電子遊戲 is not in vocab
nan is not in vocab
nan is not in vocab
nan is not in vocab
nan is not in vocab
nan is not in vocab
nan is not in vocab
nan is not in vocab
nan is not in vocab
nan is not in vocab
nan is not in vocab
nan is not in vocab
nan is not in vocab
